In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel(r"E:\Projects\UrbanHub\files\NewDataset.xlsx")
df

,ID,Green space ratio,Rotation,Main street,Sub street,Bldg Footprint,Type,Bldg Centroids x,Bldg Centroids y,Lengths,...,Aspect Ratio,Annual Solar Hours,Roof radiation - Cold,Roof radiation - Hot,SVF,Ave. UTCI - Cold,Ave. UTCI - Hot,Ave. Percent of Shaded area,Total EUI - Cold,Total EUI - Hot
0,B0,0.2,0,20,10,0.6,1,20.000000,30.666667,22.000000,...,0.9,448638,1158.21,3071.83,82.19,-0.47,37.07,9,8.653,8.695
1,B1,0.2,0,20,10,0.6,1,10.000000,30.666667,22.000000,...,0.6,299175,1034.74,2696.77,82.19,-0.47,37.07,9,8.653,8.695
2,B2,0.2,0,20,10,0.6,1,20.000000,77.333333,22.000000,...,0.6,264846,1042.35,2689.40,78.23,-0.47,37.07,9,8.653,8.695
3,B3,0.2,0,20,10,0.6,1,10.000000,77.333333,22.000000,...,0.9,431566,1153.40,3056.60,78.23,-0.47,37.07,9,8.653,8.695
4,B4,0.2,0,20,10,0.6,1,20.000000,124.000000,22.000000,...,0.6,265691,1060.30,2746.95,76.76,-0.47,37.07,9,8.653,8.695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,P0,0.2,90,25,5,0.6,0,-13.333333,103.750000,18.333333,...,0.0,0,0.00,0.00,84.17,0.01,36.59,16,8.779,10.197
15892,P1,0.2,90,25,5,0.6,0,33.333333,103.750000,18.333333,...,0.0,0,0.00,0.00,70.53,0.01,36.59,16,8.779,10.197
15893,P2,0.2,90,25,5,0.6,0,10.000000,58.750000,18.333333,...,0.0,0,0.00,0.00,62.54,0.01,36.59,16,8.779,10.197
15894,P3,0.2,90,25,5,0.6,0,33.333333,36.250000,18.333333,...,0.0,0,0.00,0.00,73.99,0.01,36.59,16,8.779,10.197


In [3]:
X = df[["Bldg Centroids x", "Bldg Centroids y","Bldg Footprint", "Type", "Lengths", "Widths", "Stories"]]
y = df[[
        "Typology", "X", "Y", "Density", "Visibility", "Cooling - Cold", "Heating - Cold", "Lighting - Cold", "Hot water - Cold",
        "Gas - Cold", "Cooling - Hot", "Heating - Hot", "Lighting - Hot", "Hot water - Hot", "Gas - Hot", "Compactness 1",
        "Shape Factor", "Aspect Ratio", "Annual Solar Hours", "Roof radiation - Cold", "Roof radiation - Hot"
        ]]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)